In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math
import hashlib
import builtins
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 48.6 MB/s 
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 59 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155225 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=JTpyDvMvsJRBxK7FCSrcA4YBPNjEmf&prompt=consent&access_type=offline&code_challenge=it_UIrPHlTWrGet2D4QuUwgmU9P1_-LcJPXzY9imdtI&code_challenge_method=S256

Enter verification code: 4/1AX4XfWgtut18v1Ozq9_4nWKza_dMlZO65c4jYyCFbgain5FSA0cucgRAEts


In [ ]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
-
Operation completed over 1 objects/316.7 MiB.                                    


In [ ]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

!mkdir text_bins
!mkdir title_bins
!mkdir anchor_bins

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [ ]:
parquetFile.count()

21084

In [ ]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text").rdd

In [ ]:

def tokenize(text):
    english_stopwords = frozenset(stopwords.words('english'))
    corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    all_stopwords = english_stopwords.union(corpus_stopwords)

    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

def df_to_dict(row):
    return (row[1], len(tokenize(row[0])))

def anchor_to_text(item):
    lst = []
    for i in item[0]:
        lst.append((i[0],i[1]))
    return lst
    
def word_count(text, id):
  
  ans_dict = {}
  ans = []
  
  for tok in tokenize(text):
    if tok in ans_dict:
      ans_dict[tok] += 1
    else:
      ans_dict[tok] = 1
  for tok, tf in ans_dict.items():
    ans.append((tok, (id, tf)))
  return ans

def reduce_word_counts(unsorted_pl):  
    return sorted(unsorted_pl)

def calculate_df(postings):
  return postings.mapValues(lambda x: len(x))

def create_term_total(item):
    term_total = 0
    for i in item:
        term_total += i[1]
    return term_total

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, filename):
  posting_partition = postings.map(lambda x: (token2bucket_id(x[0]), x))
  return posting_partition.groupByKey().map(lambda x: InvertedIndex.write_a_posting_list(x, filename))

In [ ]:
DL_rdd = doc_text_pairs.map(df_to_dict)
DL = DL_rdd.collectAsMap()


In [ ]:
## BODY
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>50)

term_total_text = dict(postings_filtered_text.mapValues(create_term_total).collect())

w2df_text = calculate_df(postings_filtered_text)
w2df_dict_text = w2df_text.collectAsMap()

posting_locs_list_text = partition_postings_and_write(postings_filtered_text, 'text_bins').collect()
super_posting_locs_text = defaultdict(list)
for posting_loc in posting_locs_list_text:
  for k, v in posting_loc.items():
    super_posting_locs_text[k].extend(v)


In [ ]:
## TITLE
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

term_total_title = dict(postings_title.mapValues(create_term_total).collect())

w2df_title = calculate_df(postings_title)
w2df_dict_title = w2df_title.collectAsMap()

posting_locs_list_title = partition_postings_and_write(postings_title, 'title_bins').collect()
super_posting_locs_title = defaultdict(list)
for posting_loc in posting_locs_list_title:
  for k, v in posting_loc.items():
    super_posting_locs_title[k].extend(v)


In [ ]:
## ANCHOR

anchor_text_pairs=doc_anchor_pairs.flatMap(anchor_to_text).distinct().groupByKey().mapValues(list)
anchor_text=anchor_text_pairs.map(lambda x:(" ".join(x[1]), x[0]))

word_counts_anchor = anchor_text.flatMap(lambda x: word_count(x[0], x[1]))

postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)

term_total_anchor = dict(postings_anchor.mapValues(create_term_total).collect())

w2df_anchor = calculate_df(postings_anchor)
w2df_dict_anchor = w2df_anchor.collectAsMap()

posting_locs_list_anchor = partition_postings_and_write(postings_anchor, 'anchor_bins').collect()
super_posting_locs_anchor = defaultdict(list)
for posting_loc in posting_locs_list_anchor:
  for k, v in posting_loc.items():
    super_posting_locs_anchor[k].extend(v)


In [ ]:
# Create inverted index instance for body
inverted_text = InvertedIndex()
inverted_text.df = w2df_dict_text
inverted_text.term_total = term_total_text
inverted_text.posting_locs = super_posting_locs_text
inverted_text.write_index('.', 'text_index')

In [ ]:
# Create inverted index instance for title
inverted_title = InvertedIndex()
inverted_title.df = w2df_dict_title
inverted_title.term_total = term_total_title
inverted_title.posting_locs = super_posting_locs_title
inverted_title.write_index('.', 'title_index')

In [ ]:
# Create inverted index instance for anchor
inverted_anchor = InvertedIndex()
inverted_anchor.df = w2df_dict_anchor
inverted_anchor.term_total = term_total_anchor
inverted_anchor.posting_locs = super_posting_locs_anchor
inverted_anchor.write_index('.', 'anchor_index')

In [ ]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [ ]:
pages_links = parquetFile.limit(1000).select("id", "anchor_text").rdd
def generate_graph(pages):
 
  def get_ids(row):
    lst = []
    for r in row:
      lst.append(r[0])
    return list(set(lst))

  def get_edges(ids):
    lst = []
    for id in ids[1]:
      lst.append((ids[0], id))
    return lst

  def get_vertices(edges):
    lst = []
    for i in edges:
      lst.append((i,))
    return lst

  ids = pages.mapValues(get_ids)
  edges = ids.flatMap(get_edges)
  vertices = edges.flatMap(get_vertices).distinct()
  return edges, vertices

In [ ]:
edges, vertices = generate_graph(pages_links)

edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.show()